# Expectation-Maximization

In this assignment we will be using the Expectation Maximization method to estimate the parameters of the same three coin experiment as in the theoretical part. We will examine the behavior of the algorithm for various combinations of parameters.

## Description of the Experiment

The following procedure generates the data for the three coin experiment.

The parameters are:

- $\lambda$ := The probability of heads on the hidden coin H.

- $p_1$ := The probability of heads on coin A.

- $p_2$ := The probability of heads on coin B.

Each of the $N$ samples is collected the following way:

- The secret coin (H) is tossed.

- If the result is heads, coin A is tossed $M$ times and the results are recorded.

- If the result is tails, coin B is tossed $M$ times and the results are recorded.

**Heads are recorded as 1.** 

**Tails are recorded as 0.**

The data is returned as an **$N \times M$** matrix, where each of the $N$ rows correspond to the trials and contains the results of the corresponding sample (generated either by coin A or by coin B).

## Description of Provided Functions

Three functions are provided for your convenience:

*  **`utils.generateData(lambda,p1,p2,N,M)`:** Performs the experiment $N$ times with coin parameters specified as argument and returns the results in a $N \times M$ matrix.


* **`utils.unknownData()`** Returns a dataset of size $N \times M$ where generation parameters are unknown.


*  **`utils.plot(data,distribution)`:** Plot a histogram of the number of heads per trial along with the probability distribution. This function will be used to visualize the progress of the EM algorithm at every iteration.

An example of use of these two functions is given below:

In [180]:
%matplotlib inline
import numpy,utils

# Print the data matrix as a result of the three coins experiment with parameter 0.5, 0.8 and 0.2.
#data = utils.generateData(0.5,0.8,0.2,15,5)
data = utils.unknownData()


#Print the data histogram along with a uniform probability distribution.
#utils.plot( data, numpy.ones([data.shape[1]+1]) / (data.shape[1]+1) )

## Calculate the Log-Likelihood (10 P)

Implement a function which calculates the log likelihood for a given dataset and parameters. The log-likelihood is given by:
$$
LL = \frac1N \sum_{i=1}^N \log \!\! \sum_{z \in \{\mathrm{heads},\mathrm{tails}\}} \!\! P(X=x_i, Z=z \mid \theta)\\
   = \frac1N \sum_{i=1}^N \log \left[ \lambda \cdot p_1^{h(x_i)} \cdot (1-p_1)^{t(x_i)} + (1-\lambda) \cdot p_2^{h(x_i)} \cdot (1-p_2)^{t(x_i)} \right]
$$
where $h(x_i)$ and $t(x_i)$ denote the number of heads and tails in sample $i$, respectively. Note that we take the averaged log-likelihood over all trials, hence the multiplicative term $\frac1N$ in front.

In [186]:
import numpy as np

def loglikelihood(data,lam,p1,p2):
    logSum = 0
    
    for i in range(data.shape[0]):
        heads = (data[i] == 1).sum()
        tails = (data[i] == 0).sum()
        
        logSum += np.log(totalThrowsH(lam, p1, p2, heads, tails))
        
    return logSum / data.shape[0]

def totalThrows(lam, p1, p2, heads, tails):
    return meanHeads(lam, p1, heads, tails) + meanHeads(1-lam, p2, heads, tails)

def meanHeads(lam, proba, heads, tails):
    return lam * (proba**heads) * (1 - proba)**tails


## Implementing and Running the EM Algorithm (30 P)

Implement a function which iteratively determines the values of $\lambda$, $p_1$ and $p_2$. The function starts with some initial estimates for the parameters and returns the results of the method for those parameters.

In each iteration, the following update rules are used for the parameters:

$$\lambda^{new} = \frac{E(\#heads(coin\_H))}{\#throws(coin\_H)} = \frac{1}{N}\sum_{i=1}^{N}\frac{\lambda p_1^{h(x_i)}(1-p_1)^{t(x_i)}}{\lambda p_1^{h(x_i)}(1-p_1)^{t(x_i)} + (1-\lambda)p_2^{h(x_i)}(1-p_2)^{t(x_i)}}$$

$$p_1^{new} = \frac{E(\#heads(coin\_A))}{E(\#throws(coin\_A))} = \frac{\sum_{i=1}^{N}R_1(i)h(x_i)}{M \sum_{i=1}^{N}R_1(i)}$$

$$p_2^{new} = \frac{E(\#heads(coin\_B))}{E(\#throws(coin\_B))} = \frac{\sum_{i=1}^{N}R_2(i)h(x_i)}{M \sum_{i=1}^{N}R_2(i)}$$

where $h(x_i)$ and $t(x_i)$ denote the number of heads and tails in sample i, respectively, and

$$R_1(i) = \frac{\lambda p_1^{h(x_i)}(1-p_1)^{t(x_i)}}{\lambda p_1^{h(x_i)}(1-p_1)^{t(x_i)} + (1-\lambda)p_2^{h(x_i)}(1-p_2)^{t(x_i)}}$$

$$R_2(i) = \frac{(1-\lambda) p_2^{h(x_i)}(1-p_2)^{t(x_i)}}{\lambda p_1^{h(x_i)}(1-p_1)^{t(x_i)} + (1-\lambda)p_2^{h(x_i)}(1-p_2)^{t(x_i)}}$$


**TODO:**

* **Implement the EM learning procedure.**
* **Use as stopping criterion the improvement of log-likelihood between two iterations to be smaller than $0.001$.**
* **Run the EM procedure on the data returned by function `utils.unknownData()`. Use as an initial solution for your model the parameters $\lambda = 0.5$, $p_1 = 0.25$, $p_2 = 0.75$ **.
* **At each iteration of the EM procedure, print the log-likelihood and the value of your model parameters, and plot the learned probability distribution using the function utils.plot().**

In [187]:
import utils
%matplotlib inline

lam = 0.5
p1  = 0.25
p2  = 0.75

criterion = False # (to be set to True)
it = 0
oldLL = loglikelihood(data,lam,p1,p2)

# Iterate until the stopping criterion is satisfied
while (criterion == False):
    it += 1
    
    #calc new lamda
    sumDiv = 0
    
    #calc new p1
    sumEnum1 = 0
    sumDenom1 = 0
    
    #calc new p2
    sumEnum2 = 0
    sumDenom2 = 0
    for i in range(data.shape[0]):
        heads = (data[i] == 1).sum()
        tails = (data[i] == 0).sum()
        
        #new lamda
        sumDiv += meanHeads(lam, p1, heads, tails) / totalThrowsH(lam, p1, p2, heads, tails)
    
        #new p1
        sumEnum1 += heads * (meanHeads(lam, p1, heads, tails) / totalThrows(lam, p1, p2, heads, tails))
        sumDenom1 += meanHeads(lam, p1, heads, tails) / totalThrows(lam, p1, p2, heads, tails)
    
        #new p2
        sumEnum2 += heads * (meanHeads(1-lam, p2, heads, tails) / totalThrows(lam, p1, p2, heads, tails))
        sumDenom2 += meanHeads(1-lam, p2, heads, tails) / totalThrows(lam, p1, p2, heads, tails)
    
    
    newLam = sumDiv / data.shape[0]
    newP1 = sumEnum1 / (data.shape[1] * sumDenom1)
    newP2 = sumEnum2 / (data.shape[1] * sumDenom2)
    ll =  loglikelihood(data,newLam,newP1,newP2)
    

    print 'it:%2d  lambda: %.2f  p1: %.2f  p2: %.2f  ll: %.3f'%(
            it, newLam, newP1, newP2, ll)
    
    #utils.plot( data, numpy.ones([data.shape[1]+1]) / (data.shape[1]+1) )
    
    if abs(abs(oldLL) - abs(ll)) < 0.001:
        criterion = True
    
    lam = newLam
    p1 = newP1
    p2 = newP2
    oldLL = ll
# -------------------------------------------

it: 1  lambda: 0.15  p1: 0.41  p2: 0.76  ll: -11.709
it: 2  lambda: 0.18  p1: 0.44  p2: 0.77  ll: -11.684
it: 3  lambda: 0.21  p1: 0.46  p2: 0.78  ll: -11.672
it: 4  lambda: 0.23  p1: 0.47  p2: 0.78  ll: -11.666
it: 5  lambda: 0.24  p1: 0.48  p2: 0.78  ll: -11.664
it: 6  lambda: 0.25  p1: 0.48  p2: 0.79  ll: -11.662
it: 7  lambda: 0.26  p1: 0.48  p2: 0.79  ll: -11.662


## More Experiments (10 P)

Examine the behaviour of the EM algorithm for various combinations of data generation parameters and initializations (for generating various distributions, use the method `utils.generateData(...)`). In particular, find settings for which:

* The role of coins $A$ and $B$ are permuted between the data generating model and the learned model (i.e. $\hat p_1 \approx p_2$, $\hat p_2 \approx p_1$ and $\hat \lambda \approx 1-\lambda$).

* The EM procedure takes a long time to converge.

Print the parameters and log-likelihood objective at each iteration. Only display the plot for the converged model.